# General Linear Elliptic Equation

The `GeneralMG2d` class implements support for a general elliptic equation of the form:

$$\alpha \phi + \nabla \cdot (\beta \nabla \phi) + \gamma \cdot \nabla \phi = f$$

with inhomogeneous boundary conditions.

It subclasses the `CellCenterMG2d` class, and the basic interface is the same

We will solve the above with

\begin{align}
\alpha &= 10 \\
\beta &= xy + 1 \\
\gamma &= \hat{x} + \hat{y}
\end{align}

and

\begin{align}
f = &-\frac{\pi}{2}(x + 1)\sin\left( \frac{\pi y}{2}\right) \cos\left(\frac{\pi x}{2}\right )  \\
{}  &-\frac{\pi}{2}(y + 1)\sin\left( \frac{\pi x}{2}\right) \cos\left(\frac{\pi y}{2}\right )  \\
{}  &+ \left( \frac{-\pi^2 (xy+1)}{2} + 10\right ) \cos \left ( \frac{\pi x}{2} \right ) \cos\left ( \frac{\pi y}{2} \right )
\end{align}


on $[0, 1] \times [0,1]$ with boundary conditions:

\begin{align}
\phi(x=0) &= \cos(\pi y/2) \\
\phi(x=1) &= 0 \\
\phi(y=0) &= \cos(\pi x/2) \\
\phi(y=1) &= 0
\end{align}


This has the exact solution:

$$\phi = \cos(\pi x/2) \cos(\pi y/2)$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Setting up the solver

In [2]:
import pyro.mesh.boundary as bnd
import pyro.mesh.patch as patch
import pyro.multigrid.general_MG as gMG

For reference, we'll define a function providing the analytic solution

In [3]:
def true(x,y):                                                                                         
    return np.cos(np.pi*x/2.0)*np.cos(np.pi*y/2.0)                                                     

Now the coefficents--note that since $\gamma$ is a vector, we have a different function for each component

In [4]:
def alpha(x,y):                                                                                        
    return 10.0*np.ones_like(x)                                                                        
                                                                                                       
def beta(x,y):                                                                                         
    return x*y + 1.0                                                                                   
                                                                                                       
def gamma_x(x,y):                                                                                      
    return np.ones_like(x)                                                                             
                                                                                                       
def gamma_y(x,y):                                                                                      
    return np.ones_like(x)                                                                             

and the righthand side function

In [5]:
def f(x,y):
    return -0.5*np.pi*(x + 1.0)*np.sin(np.pi*y/2.0)*np.cos(np.pi*x/2.0) - \
            0.5*np.pi*(y + 1.0)*np.sin(np.pi*x/2.0)*np.cos(np.pi*y/2.0) + \
            (-np.pi**2*(x*y+1.0)/2.0 + 10.0)*np.cos(np.pi*x/2.0)*np.cos(np.pi*y/2.0)

Our inhomogeneous boundary conditions require a function that can be evaluated on the boundary to give the value

In [6]:
def xl_func(y):
    return np.cos(np.pi*y/2.0)

def yl_func(x):
    return np.cos(np.pi*x/2.0)

Now we can setup our grid object and the coefficients, which are stored as a `CellCenter2d` object.  Note, the coefficients do not need to have the same boundary conditions as $\phi$ (and for real problems, they may not).  The one that matters the most is $\beta$, since that will need to be averaged to the edges of the domain, so the boundary conditions on the coefficients are important.

Here we use Neumann boundary conditions

In [7]:
nx = 128
ny = 128

g = patch.Grid2d(nx, ny, ng=1)
d = patch.CellCenterData2d(g)

bc_c = bnd.BC(xlb="neumann", xrb="neumann",
              ylb="neumann", yrb="neumann")

d.register_var("alpha", bc_c)
d.register_var("beta", bc_c)
d.register_var("gamma_x", bc_c)
d.register_var("gamma_y", bc_c)
d.create()

a = d.get_var("alpha")
a[:,:] = alpha(g.x2d, g.y2d)

b = d.get_var("beta")
b[:,:] = beta(g.x2d, g.y2d)

gx = d.get_var("gamma_x")
gx[:,:] = gamma_x(g.x2d, g.y2d)

gy = d.get_var("gamma_y")
gy[:,:] = gamma_y(g.x2d, g.y2d)

Now we can setup the multigrid object

In [8]:
a = gMG.GeneralMG2d(nx, ny,
                    xl_BC_type="dirichlet", yl_BC_type="dirichlet",
                    xr_BC_type="dirichlet", yr_BC_type="dirichlet",
                    xl_BC=xl_func,
                    yl_BC=yl_func,
                    coeffs=d,
                    verbose=1, vis=0, true_function=true)

cc data: nx = 2, ny = 2, ng = 1
         nvars = 7
         variables:
               v: min:    0.0000000000    max:    0.0000000000
                  BCs: -x: dirichlet    +x: dirichlet    -y: dirichlet    +y: dirichlet   
               f: min:    0.0000000000    max:    0.0000000000
                  BCs: -x: dirichlet    +x: dirichlet    -y: dirichlet    +y: dirichlet   
               r: min:    0.0000000000    max:    0.0000000000
                  BCs: -x: dirichlet    +x: dirichlet    -y: dirichlet    +y: dirichlet   
           alpha: min:    0.0000000000    max:    0.0000000000
                  BCs: -x: neumann      +x: neumann      -y: neumann      +y: neumann     
            beta: min:    0.0000000000    max:    0.0000000000
                  BCs: -x: neumann      +x: neumann      -y: neumann      +y: neumann     
         gamma_x: min:    0.0000000000    max:    0.0000000000
                  BCs: -x: neumann      +x: neumann      -y: neumann      +y: neumann     
     

just as before, we specify the righthand side and initialize the solution

In [9]:
a.init_zeros()
a.init_RHS(f(a.x2d, a.y2d))

Source norm =  1.775181492337501


## Solving the system

In [10]:
a.solve(rtol=1.e-10)

source norm =  1.775181492337501
<<< beginning V-cycle (cycle 1) >>>

  level =  6, nx =  128, residual change:     1.77518 →     188.933
  level =  5, nx =   64, residual change:     129.938 →     56.2871
  level =  4, nx =   32, residual change:     38.8869 →     18.7228
  level =  3, nx =   16, residual change:     12.9261 →     6.74186
  level =  2, nx =    8, residual change:     4.64648 →     2.06513
  level =  1, nx =    4, residual change:     1.37453 →   0.0224452
  bottom solve
  level =  1, nx =    4, residual change:   0.0312525 →   0.0312525
  level =  2, nx =    8, residual change:     2.80598 →     2.80598
  level =  3, nx =   16, residual change:      8.7724 →      8.7724
  level =  4, nx =   32, residual change:      19.591 →      19.591
  level =  5, nx =   64, residual change:     50.4641 →     50.4641
  level =  6, nx =  128, residual change:     160.213 →     160.213
cycle 1: relative err = 0.9999999999999981, residual err = 2.323786088373021

<<< beginning V-cycle

## Checking the result

We can compare to the true solution

In [11]:
v = a.get_solution()
b = true(a.x2d, a.y2d)
e = v - b

The norm of the error is

In [12]:
print(f"{e.norm():20.10g}")

     1.671934405e-05
